In [8]:
import pandas as pd
import numpy as np
from scipy import interpolate
from src.online_learning.cluster import Cluster

In [9]:
# Load the Excel file with all sheets
file_path1 = 'initialization/DT_CU1_parameters_1_exp.xlsx'  
all_sheets = pd.read_excel(file_path1, sheet_name=None)

# Check the sheet names
print(all_sheets.keys())

dict_keys(['T = 25', 'T = 10', 'T = 40'])


In [10]:
df_list = []
for sheet_name, df in all_sheets.items():
    print(f"Sheet name: {sheet_name}")
    df_list.append(df)

Sheet name: T = 25
Sheet name: T = 10
Sheet name: T = 40


In [11]:
df_T_25 = df_list[0]
df_T_10 = df_list[1]
df_T_40 = df_list[2]

In [14]:
columns_to_drop = ['filename', 'data', 'start interval', 'end interval',
                   'tau_guess','num value extrap',
                   'R squared', 'OCV', 'tau_opt']

In [15]:
df_T_25 = df_T_25.drop(columns_to_drop, axis=1)
df_T_10 = df_T_10.drop(columns_to_drop, axis=1)
df_T_40 = df_T_40.drop(columns_to_drop, axis=1)

In [16]:
df_T_25 = df_T_25.dropna()
df_T_10 = df_T_10.dropna()
df_T_40 = df_T_40.dropna()

In [17]:
df_T_25['Temp'] = 25
df_T_10['Temp'] = 10
df_T_40['Temp'] = 40

In [18]:
df_T_25.describe()

,Soc,R_0,R_1,C_1,Temp
count,55.000000,55.000000,55.000000,55.000000,55.0
mean,50.000000,0.011520,0.015499,11965.948149,25.0
std,31.914237,0.002412,0.006471,3143.296922,0.0
min,0.000000,0.009210,0.009238,4919.388300,25.0
25%,20.000000,0.010363,0.011560,10500.078051,25.0
50%,50.000000,0.010981,0.013876,11751.825878,25.0
75%,80.000000,0.011748,0.016568,13995.291319,25.0
max,100.000000,0.022375,0.041071,19489.447086,25.0


In [19]:
df_T_25.describe()

,Soc,R_0,R_1,C_1,Temp
count,55.000000,55.000000,55.000000,55.000000,55.0
mean,50.000000,0.011520,0.015499,11965.948149,25.0
std,31.914237,0.002412,0.006471,3143.296922,0.0
min,0.000000,0.009210,0.009238,4919.388300,25.0
25%,20.000000,0.010363,0.011560,10500.078051,25.0
50%,50.000000,0.010981,0.013876,11751.825878,25.0
75%,80.000000,0.011748,0.016568,13995.291319,25.0
max,100.000000,0.022375,0.041071,19489.447086,25.0


In [24]:
df_T_10_soc_one = df_T_10[( df_T_10['Soc'] >= 0) & (df_T_10['Soc'] <= 50) ]
df_T_25_soc_one = df_T_25[ ( df_T_25['Soc'] >= 0) & (df_T_25['Soc'] <= 50) ]
df_T_40_soc_one = df_T_40[ ( df_T_40['Soc'] >= 0) & (df_T_40['Soc'] <= 50) ]

chunk_size = 6
temp = np.array([10, 25])
soc = np.array([0, 10, 20, 30, 40, 50])
parameters = ['R_0', 'R_1', 'C_1']
temp_new = [15, 20]

# List to store results for each parameter
interpolated_results = []

for start in range(0, len(df_T_10_soc_one), chunk_size):
    chunk_10 = df_T_10_soc_one.iloc[start:start + chunk_size]
    chunk_25 = df_T_25_soc_one.iloc[start:start + chunk_size]
    
    for param in parameters:
        # Extracting the parameter values for the chunks
        values_10 = chunk_10[param].values
        values_25 = chunk_25[param].values
        
        # Check the shape of values_10 and values_25 to make sure they match soc.size
        if len(values_10) != len(soc) or len(values_25) != len(soc):
            print(f"Chunk size mismatch for parameter {param} at index {start}")
            continue
        
        # Creating the array for interpolation with the correct shape (n, m)
        z = np.vstack((values_10, values_25)).T  # Transpose to get shape (n, m)
        
        # Performing the interpolation
        f = interpolate.interp2d(temp, soc, z, kind='linear')
        z_new = f(temp_new, soc)
        
        # Store the interpolated results
        for i, temp_val in enumerate(temp_new):
            for j, soc_val in enumerate(soc):
                interpolated_results.append({
                    'temp': temp_val,
                    'soc': soc_val,
                    'param': param,
                    'value': z_new[j, i]
                })

# Convert the interpolated results to a DataFrame
df_interpolated = pd.DataFrame(interpolated_results)

# Pivot the DataFrame to get the desired structure
df_interpolated_pivot = df_interpolated.pivot_table(index=['temp', 'soc'], columns='param', values='value').reset_index()

print(df_interpolated_pivot)

param  temp  soc           C_1       R_0       R_1
0        15    0   9362.838793  0.016177  0.016432
1        15   10   9691.182929  0.016623  0.019499
2        15   20  12052.351794  0.017420  0.023310
3        15   30   9306.629723  0.018662  0.024055
4        15   40   5139.236488  0.024588  0.035098
5        15   50   2142.033750  0.025116  0.108859
6        20    0  11140.809709  0.013595  0.014089
7        20   10  11096.331372  0.014090  0.016753
8        20   20  13364.479790  0.014450  0.019963
9        20   30  11500.390560  0.015292  0.019551
10       20   40   8387.385488  0.018321  0.025878
11       20   50   4165.597126  0.020244  0.070274


/tmp/ipykernel_7047/4264066699.py:36: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://gist.github.com/ev-br/8544371b40f414b7eaf3fe6217209bff`

  f = interpolate.interp2d(temp, soc, z, kind='linear')
/tmp/ipykernel_7047/4264066699.py:37: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular gr

In [27]:
df_one = pd.concat([df_interpolated_pivot,df_T_10_soc_one,df_T_25_soc_one])

In [41]:
cluster_one = [np.array(row) for row in df_one.to_numpy()]
len(cluster_one)

60

## cell2

In [29]:
df_T_10_soc_two = df_T_10[( df_T_10['Soc'] >= 50) & (df_T_10['Soc'] <= 100) ]
df_T_25_soc_two = df_T_25[ ( df_T_25['Soc'] >= 50) & (df_T_25['Soc'] <= 100) ]
#df_T_40_soc_one = df_T_40[ ( df_T_40['Soc'] >= 0) & (df_T_40['Soc'] <= 50) ]

chunk_size = 6
temp = np.array([10, 25])
soc = np.array([0, 10, 20, 30, 40, 50])
parameters = ['R_0', 'R_1', 'C_1']
temp_new = [15, 20]

# List to store results for each parameter
interpolated_results = []

for start in range(0, len(df_T_10_soc_two), chunk_size):
    chunk_10 = df_T_10_soc_two.iloc[start:start + chunk_size]
    chunk_25 = df_T_25_soc_one.iloc[start:start + chunk_size]
    
    for param in parameters:
        # Extracting the parameter values for the chunks
        values_10 = chunk_10[param].values
        values_25 = chunk_25[param].values
        
        # Check the shape of values_10 and values_25 to make sure they match soc.size
        if len(values_10) != len(soc) or len(values_25) != len(soc):
            print(f"Chunk size mismatch for parameter {param} at index {start}")
            continue
        
        # Creating the array for interpolation with the correct shape (n, m)
        z = np.vstack((values_10, values_25)).T  # Transpose to get shape (n, m)
        
        # Performing the interpolation
        f = interpolate.interp2d(temp, soc, z, kind='linear')
        z_new = f(temp_new, soc)
        
        # Store the interpolated results
        for i, temp_val in enumerate(temp_new):
            for j, soc_val in enumerate(soc):
                interpolated_results.append({
                    'temp': temp_val,
                    'soc': soc_val,
                    'param': param,
                    'value': z_new[j, i]
                })

# Convert the interpolated results to a DataFrame
df_interpolated_two = pd.DataFrame(interpolated_results)

# Pivot the DataFrame to get the desired structure
df_interpolated_pivot_two = df_interpolated_two.pivot_table(index=['temp', 'soc'], columns='param', values='value').reset_index()

print(df_interpolated_pivot_two)

param  temp  soc           C_1       R_0       R_1
0        15    0   9655.887330  0.013612  0.014383
1        15   10   8960.577385  0.014149  0.014264
2        15   20   9455.065286  0.014868  0.018994
3        15   30  14273.722858  0.015673  0.021401
4        15   40   8889.953426  0.015858  0.017680
5        15   50   7119.632085  0.017630  0.023079
6        20    0  11287.333977  0.012312  0.013065
7        20   10  10731.028600  0.012853  0.014136
8        20   20  12065.836535  0.013174  0.017804
9        20   30  13983.937127  0.013798  0.018224
10       20   40  10262.743957  0.013956  0.017169
11       20   50   6654.396293  0.016500  0.027384


/tmp/ipykernel_7047/2706429875.py:32: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://gist.github.com/ev-br/8544371b40f414b7eaf3fe6217209bff`

  f = interpolate.interp2d(temp, soc, z, kind='linear')
/tmp/ipykernel_7047/2706429875.py:33: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular gr

In [38]:
cluster_two = pd.concat([df_interpolated_two,df_T_10_soc_two,df_T_25_soc_two])

In [31]:
len(cluster_two)

156

## cell3

In [47]:
#df_T_10_soc_one = df_T_10[( df_T_10['Soc'] >= 0) & (df_T_10['Soc'] <= 50) ]
df_T_25_soc_three = df_T_25[ ( df_T_25['Soc'] >= 0) & (df_T_25['Soc'] <= 50) ]
df_T_40_soc_three = df_T_40[ ( df_T_40['Soc'] >= 0) & (df_T_40['Soc'] <= 50) ]

chunk_size = 6
temp = np.array([25, 40])
soc = np.array([0, 10, 20, 30, 40, 50])
parameters = ['R_0', 'R_1', 'C_1']
temp_new = [30, 35]

# List to store results for each parameter
interpolated_results = []

for start in range(0, len(df_T_10_soc_one), chunk_size):
    chunk_25 = df_T_25_soc_one.iloc[start:start + chunk_size]
    chunk_40 = df_T_40_soc_one.iloc[start:start + chunk_size]
    
    for param in parameters:
        # Extracting the parameter values for the chunks
        values_25 = chunk_25[param].values
        values_40 = chunk_40[param].values
        
        # Check the shape of values_10 and values_25 to make sure they match soc.size
        if len(values_25) != len(soc) or len(values_40) != len(soc):
            print(f"Chunk size mismatch for parameter {param} at index {start}")
            continue
        
        # Creating the array for interpolation with the correct shape (n, m)
        z = np.vstack((values_25, values_40)).T  # Transpose to get shape (n, m)
        
        # Performing the interpolation
        f = interpolate.interp2d(temp, soc, z, kind='linear')
        z_new = f(temp_new, soc)
        
        # Store the interpolated results
        for i, temp_val in enumerate(temp_new):
            for j, soc_val in enumerate(soc):
                interpolated_results.append({
                    'temp': temp_val,
                    'soc': soc_val,
                    'param': param,
                    'value': z_new[j, i]
                })

# Convert the interpolated results to a DataFrame
df_interpolated_three = pd.DataFrame(interpolated_results)

# Pivot the DataFrame to get the desired structure
df_interpolated_pivot_three = df_interpolated.pivot_table(index=['temp', 'soc'], columns='param', values='value').reset_index()

print(df_interpolated_pivot_three)

param  temp  soc           C_1       R_0       R_1
0        30    0  13670.965302  0.009951  0.010635
1        30   10  12734.187684  0.010379  0.012775
2        30   20  15893.233768  0.010449  0.014999
3        30   30  15151.324276  0.010657  0.013196
4        30   40  15778.581428  0.010933  0.014324
5        30   50  14611.322527  0.013400  0.026787
6        35    0  14423.149979  0.008890  0.009523
7        35   10  12966.895552  0.009201  0.011542
8        35   20  17109.859752  0.009417  0.013384
9        35   30  16608.497155  0.009392  0.011345
10       35   40  19921.628368  0.009812  0.011990
11       35   50  23033.484553  0.011429  0.021886


/tmp/ipykernel_7047/1940777348.py:32: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://gist.github.com/ev-br/8544371b40f414b7eaf3fe6217209bff`

  f = interpolate.interp2d(temp, soc, z, kind='linear')
/tmp/ipykernel_7047/1940777348.py:33: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular gr

In [48]:
cluster_three = pd.concat([df_interpolated_pivot_three, df_T_40_soc_three ])

In [49]:
len(cluster_three)

30

## cell4

In [50]:
#df_T_10_soc_two = df_T_10[( df_T_10['Soc'] >= 50) & (df_T_10['Soc'] <= 100) ]
df_T_25_soc_four = df_T_25[ ( df_T_25['Soc'] >= 50) & (df_T_25['Soc'] <= 100) ]
df_T_40_soc_four = df_T_40[ ( df_T_40['Soc'] >= 50) & (df_T_40['Soc'] <= 100) ]

chunk_size = 6
temp = np.array([25, 40])
soc = np.array([0, 10, 20, 30, 40, 50])
parameters = ['R_0', 'R_1', 'C_1']
temp_new = [30, 35]

# List to store results for each parameter
interpolated_results = []

for start in range(0, len(df_T_10_soc_two), chunk_size):
    chunk_25 = df_T_25_soc_four.iloc[start:start + chunk_size]
    chunk_40 = df_T_40_soc_four.iloc[start:start + chunk_size]
    
    for param in parameters:
        # Extracting the parameter values for the chunks
        values_25 = chunk_25[param].values
        values_40 = chunk_40[param].values
        
        # Check the shape of values_10 and values_25 to make sure they match soc.size
        if len(values_25) != len(soc) or len(values_40) != len(soc):
            print(f"Chunk size mismatch for parameter {param} at index {start}")
            continue
        
        # Creating the array for interpolation with the correct shape (n, m)
        z = np.vstack((values_25, values_40)).T  # Transpose to get shape (n, m)
        
        # Performing the interpolation
        f = interpolate.interp2d(temp, soc, z, kind='linear')
        z_new = f(temp_new, soc)
        
        # Store the interpolated results
        for i, temp_val in enumerate(temp_new):
            for j, soc_val in enumerate(soc):
                interpolated_results.append({
                    'temp': temp_val,
                    'soc': soc_val,
                    'param': param,
                    'value': z_new[j, i]
                })

# Convert the interpolated results to a DataFrame
df_interpolated_four = pd.DataFrame(interpolated_results)

# Pivot the DataFrame to get the desired structure
df_interpolated_pivot_four = df_interpolated_two.pivot_table(index=['temp', 'soc'], columns='param', values='value').reset_index()

print(df_interpolated_pivot_four)

param  temp  soc           C_1       R_0       R_1
0        15    0   9655.887330  0.013612  0.014383
1        15   10   8960.577385  0.014149  0.014264
2        15   20   9455.065286  0.014868  0.018994
3        15   30  14273.722858  0.015673  0.021401
4        15   40   8889.953426  0.015858  0.017680
5        15   50   7119.632085  0.017630  0.023079
6        20    0  11287.333977  0.012312  0.013065
7        20   10  10731.028600  0.012853  0.014136
8        20   20  12065.836535  0.013174  0.017804
9        20   30  13983.937127  0.013798  0.018224
10       20   40  10262.743957  0.013956  0.017169
11       20   50   6654.396293  0.016500  0.027384


/tmp/ipykernel_7047/3997654561.py:32: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://gist.github.com/ev-br/8544371b40f414b7eaf3fe6217209bff`

  f = interpolate.interp2d(temp, soc, z, kind='linear')
/tmp/ipykernel_7047/3997654561.py:33: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular gr

In [51]:
cluster_four = pd.concat([df_interpolated_pivot_four, df_T_40_soc_four])

In [52]:
len(cluster_four)

30